Make a python script that can automate the process for creating publication entries. If possible, the script should get the author names, article title, journal name, publication date, and abstract and compile it into a templated markdown file for insertion into the publications page.

For the publications: we can stick with pubs starting from 2018 or 2019 since that's when the lab started up. There will be a relatively large number of publications here, so it would probably be ideal if this could be automated to some degree (like a script that could automatically parse a google scholar entry or DOI into a properly formatted markdown file that matches the template for the wowchemy publications page.

In [1]:
# credit to https://medium.com/@nandinisaini021/scraping-publications-of-aerial-image-research-papers-on-google-scholar-using-python-a0dee9744728
# for the majority of the logic
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from time import sleep

In [9]:
# this function for the getting inforamtion of the web page
def get_paperinfo(paper_url):
    
  headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}

  #download the page
  response=requests.get(url,headers=headers)

  # check successful response
  if response.status_code != 200:
    print('Status code:', response.status_code)
    raise Exception('Failed to fetch web page ')

  #parse using beautiful soup
  paper_doc = BeautifulSoup(response.text,'html.parser')

  return paper_doc

In [24]:
# this function for the extracting information of the tags
def get_tags(doc):
  paper_tag = doc.select('[data-lid]')
  # cite_tag = doc.select('[title=Cite] + a')
  link_tag = doc.find_all('h3',{"class" : "gs_rt"})
  author_tag = doc.find_all("div", {"class": "gs_a"})

  return paper_tag,link_tag,author_tag

In [11]:
# it will return the title of the paper
def get_papertitle(paper_tag):
  
  paper_names = []
  
  for tag in paper_tag:
    paper_names.append(tag.select('h3')[0].get_text())

  return paper_names

In [25]:
# it will return the number of citation of the paper
# def get_citecount(cite_tag):
#   cite_count = []
#   for i in cite_tag:
#     cite = i.text
#     if i is None or cite is None:  # if paper has no citatation then consider 0
#       cite_count.append(0)
#     else:
#       tmp = re.search(r'\d+', cite) # its handle the None type object error and re use to remove the string " cited by " and return only integer value
#       if tmp is None :
#         cite_count.append(0)
#       else :
#         cite_count.append(int(tmp.group()))

#   return cite_count

In [13]:
# function for the getting link information
def get_link(link_tag):

  links = []

  for i in range(len(link_tag)) :
    links.append(link_tag[i].a['href']) 

  return links 

In [26]:
# function for the getting autho , year and publication information
def get_author_year_publi_info(authors_tag):
  years = []
  publication = []
  authors = []
  for i in range(len(authors_tag)):
      authortag_text = (authors_tag[i].text).split()
      year = int(re.search(r'\d+', authors_tag[i].text).group())
      years.append(year)
      publication.append(authortag_text[-1])
      author = authortag_text[0] + ' ' + re.sub(',','', authortag_text[1])
      authors.append(author)
  
  return years , publication, authors

In [29]:
# creating final repository
paper_repos_dict = {
                    'Paper Title' : [],
                    'Year' : [],
                    'Author' : [],
                    'Publication' : [],
                    'Url of paper' : [] }

# adding information in repository
def add_in_paper_repo(papername,year,author,publi,link):
  paper_repos_dict['Paper Title'].extend(papername)
  paper_repos_dict['Year'].extend(year)
  paper_repos_dict['Author'].extend(author)
  # paper_repos_dict['Citation'].extend(cite)
  paper_repos_dict['Publication'].extend(publi)
  paper_repos_dict['Url of paper'].extend(link)
    
  print(len(paper_repos_dict['Paper Title']))
  print(len(paper_repos_dict['Year']))
  print(len(paper_repos_dict['Author']))
  # print(len(paper_repos_dict['Citation']))
  print(len(paper_repos_dict['Publication']))
  print(len(paper_repos_dict['Url of paper']))

  return pd.DataFrame(paper_repos_dict)

In [30]:
for i in range (0,10,10):

  # get url for the each page
  url = "https://scholar.google.com/scholar?start={}&q=object+detection+in+aerial+image+&hl=en&as_sdt=0,5".format(i)

  # function for the get content of each page
  doc = get_paperinfo(url)

  # function for the collecting tags
  paper_tag,link_tag,author_tag = get_tags(doc)
  
  # paper title from each page
  papername = get_papertitle(paper_tag)

  # year , author , publication of the paper
  year , publication , author = get_author_year_publi_info(author_tag)

  # cite count of the paper 
  # cite = get_citecount(cite_tag)

  # url of the paper
  link = get_link(link_tag)

  # add in paper repo dict
  final = add_in_paper_repo(papername,year,author,publication,link)
  
  # use sleep to avoid status code 429
  sleep(30)

10
10
10
10
10


NameError: name 'sleep' is not defined

In [31]:
final[:10]

,Paper Title,Year,Author,Publication,Url of paper
0,DOTA: A large-scale dataset for object detecti...,2018,GS Xia,openaccess.thecvf.com,http://openaccess.thecvf.com/content_cvpr_2018...
1,Object detection in aerial images: A large-sca...,2021,J Ding,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...
2,Convolutional neural network based automatic o...,2016,I Ševo,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...
3,Clustered object detection in aerial images,2019,F Yang,openaccess.thecvf.com,http://openaccess.thecvf.com/content_ICCV_2019...
4,Learning RoI transformer for oriented object d...,2019,J Ding,openaccess.thecvf.com,http://openaccess.thecvf.com/content_CVPR_2019...
5,Tiny object detection in aerial images,2020,J Wang,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...
6,Convolutional neural networks for object detec...,2020,Y Pi,Elsevier,https://www.sciencedirect.com/science/article/...
7,Oriented object detection in aerial images wit...,2021,J Yi,openaccess.thecvf.com,http://openaccess.thecvf.com/content/WACV2021/...
8,Density map guided object detection in aerial ...,2020,C Li,openaccess.thecvf.com,http://openaccess.thecvf.com/content_CVPRW_202...
9,GSDet: Object detection in aerial images based...,2021,W Li,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...


In [ ]:
# need author names, article title, journal name, publication date, and abstract